In [1]:
# Based on this paper - https://arxiv.org/abs/1409.0473
#!pip install torch torchtext sentencepiece datasets

In [2]:
import numpy as np
import torch
from torch import nn
import functorch
import sys
import os
import math
sys.path.append(os.path.abspath("../../data"))
sys.path.append(os.path.abspath("../../nnets"))
from net_utils import get_module_list

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 8
SP_VOCAB_SIZE = 1000
TRAIN_SIZE = 500

In [3]:
from wrapper import OpusBooksDataset
from padding import PaddingSampler, pad_collate
from torch.utils.data import DataLoader

train_wrapper = OpusBooksDataset(tokenizer_vocab=SP_VOCAB_SIZE, download_split_pct="5%")
train_dataset = train_wrapper.process_dataset()
train_dataset = train_dataset.train_test_split(test_size=0.1)

train_split = train_dataset["train"]
train = DataLoader(train_split, batch_size=BATCH_SIZE, sampler=PaddingSampler(train_split["en_lens"]), collate_fn=pad_collate)

valid_split = train_dataset["test"]
valid = DataLoader(valid_split, batch_size=BATCH_SIZE, sampler=PaddingSampler(valid_split["en_lens"]), collate_fn=pad_collate)

Found cached dataset opus_books (/Users/vik/.cache/huggingface/datasets/opus_books/en-es/1.0.0/e8f950a4f32dc39b7f9088908216cd2d7e21ac35f893d04d39eb594746af2daf)
Loading cached processed dataset at /Users/vik/.cache/huggingface/datasets/opus_books/en-es/1.0.0/e8f950a4f32dc39b7f9088908216cd2d7e21ac35f893d04d39eb594746af2daf/cache-a403c05864ae83ef.arrow
Loading cached processed dataset at /Users/vik/.cache/huggingface/datasets/opus_books/en-es/1.0.0/e8f950a4f32dc39b7f9088908216cd2d7e21ac35f893d04d39eb594746af2daf/cache-6d17671bd5b4ff4f.arrow
Loading cached processed dataset at /Users/vik/.cache/huggingface/datasets/opus_books/en-es/1.0.0/e8f950a4f32dc39b7f9088908216cd2d7e21ac35f893d04d39eb594746af2daf/cache-f687e514834db6c4.arrow
Loading cached processed dataset at /Users/vik/.cache/huggingface/datasets/opus_books/en-es/1.0.0/e8f950a4f32dc39b7f9088908216cd2d7e21ac35f893d04d39eb594746af2daf/cache-56482197c2400b4f.arrow
Loading cached processed dataset at /Users/vik/.cache/huggingface/datas

In [4]:
class GRUCell(nn.Module):
    def __init__(self, input_units, hidden_units, output_units):
        super(GRUCell, self).__init__()
        self.input_units = input_units
        self.hidden_units = hidden_units
        self.output_units = output_units

        k = math.sqrt(1/hidden_units)
        self.input_weights = nn.Parameter(torch.rand(3, input_units, hidden_units) * 2 * k - k)

        self.hidden_weights = nn.Parameter(torch.rand(3, hidden_units, hidden_units) * 2 * k - k)
        self.hidden_biases = nn.Parameter(torch.rand(3, 1, hidden_units) * 2 * k - k)

    def forward(self, x, prev_hidden):
        # Compute the regular RNN forward pass
        # Compute update and reset gates for GRU
        reset_gate = torch.sigmoid(x @ self.input_weights[0,] + prev_hidden @ self.hidden_weights[0,] + self.hidden_biases[0,])
        update_gate = torch.sigmoid(x @ self.input_weights[1,] + prev_hidden @ self.hidden_weights[1,] + self.hidden_biases[1,])
        new_gate = torch.tanh(x @ self.input_weights[2,] + torch.mul(reset_gate, prev_hidden) @ self.hidden_weights[2,] + self.hidden_biases[2,])

        hidden_x = torch.mul((1 - update_gate), prev_hidden) + torch.mul(update_gate, new_gate)
        return hidden_x

In [5]:
class EncoderDecoder(nn.Module):
    def __init__(self, embedding_len, hidden_units=512, layers=2, bidirectional=True):
        super(EncoderDecoder, self).__init__()
        self.hidden_units = hidden_units
        # Bidirectional rnn needs 2x the params
        if bidirectional:
            self.context_units = self.hidden_units * 2
        else:
            self.context_units = self.hidden_units

        self.bidirectional = bidirectional
        self.embedding_len = embedding_len
        self.layers = layers

        self.embedding = nn.Embedding(embedding_len, hidden_units)
        self.fwd_encoders = get_module_list(layers, GRUCell, input_units=hidden_units, hidden_units=hidden_units, output_units=hidden_units)
        self.bwd_encoders = get_module_list(layers, GRUCell, input_units=hidden_units, hidden_units=hidden_units, output_units=hidden_units)
        self.decoders = get_module_list(layers, GRUCell, input_units=hidden_units + self.context_units, hidden_units=hidden_units, output_units=hidden_units)

        self.linear = nn.Linear(in_features=hidden_units, out_features=embedding_len)

        k = math.sqrt(1/hidden_units)
        self.hidden_attention_weight = nn.Parameter(torch.rand(hidden_units, hidden_units) * 2 * k - k)
        self.context_attention_weight = nn.Parameter(torch.rand(self.context_units, hidden_units) * 2 * k - k)
        self.attention_weight = nn.Parameter(torch.rand(1, hidden_units) * 2 * k - k)
        self.batched_diag = functorch.vmap(torch.diag)

    def attention(self, context, prev_hidden, batch_size):
        # Swap axes so the first dimension of context_attn is batch
        context_attn = torch.bmm(context.swapaxes(0,1), self.context_attention_weight.unsqueeze(0).expand(batch_size,-1,-1))
        # Swap back since prev_hidden is by batch.  This makes the first dim of cross sequence
        cross = torch.tanh(context_attn.swapaxes(0,1) + prev_hidden @ self.hidden_attention_weight)
        # This will be of dimension batch, sequence_length, 1
        attention = torch.bmm(cross.swapaxes(0,1), self.attention_weight.T.unsqueeze(0).expand(batch_size, -1, -1))
        # Drop the last singleton dimension
        attention = attention.squeeze(2)
        # Softmax the predictions across each batch
        probs = torch.softmax(attention, 1)
        diagonalized_probs = self.batched_diag(probs)
        positional_contexts = torch.sum(torch.bmm(diagonalized_probs, context.swapaxes(0,1)), dim=1).reshape(batch_size, self.context_units)
        return positional_contexts

    def generate_context(self, embedded, batch_size, encoders):
        # Encode the input sequence
        # Both tensors will have sequence then batch
        enc_hiddens = torch.zeros((1, self.layers, batch_size, self.hidden_units), device=DEVICE)
        for j in range(embedded.shape[0]):
            seq_enc_hiddens = embedded[j,:].unsqueeze(0)
            for i in range(self.layers):
                hidden = encoders[i](seq_enc_hiddens[i,], enc_hiddens[j,i])
                # Add first sequence axis
                hidden = hidden.unsqueeze(0)
                seq_enc_hiddens = torch.cat((seq_enc_hiddens, hidden), dim=0)

            enc_hiddens = torch.cat((enc_hiddens, seq_enc_hiddens[1:].unsqueeze(0)), dim=0)

        # Decode to the output sequence
        # Pass in context
        context = enc_hiddens[1:,-1,:,:]

        return context

    def forward(self, x, y, max_sequence_len):
        batch_size = x.shape[0]
        # Move batch to the second dimension, so sequence comes first
        y = y.swapaxes(0,1)
        # Embed the input sequence to reduce dimensionality
        embedded = self.embedding(x).swapaxes(0,1)

        # Generate forward context (go through input sequence from the start)
        forward_context = self.generate_context(embedded, batch_size, self.fwd_encoders)

        if self.bidirectional:
            # Generate backward context (go through input sequence from the end)
            backward_context = torch.flip(self.generate_context(torch.flip(embedded, [0]), batch_size, self.bwd_encoders), [0])
            context = torch.cat((forward_context, backward_context), dim=2)
        else:
            context = forward_context

        # Both tensors will have the first dimension be the sequence
        dec_hiddens = torch.zeros(1, self.layers, batch_size, self.hidden_units, device=DEVICE)
        outputs = torch.zeros(1, batch_size, self.embedding_len, device=DEVICE)
        for j in range(max_sequence_len):
            # Use either the actual previous y (from the input), or the generated y if the input sequence is shorter than the generation steps.
            if y.shape[0] > j:
                prev_y = y[j,:]
            else:
                prev_y = outputs[j,:,:]
                prev_y = prev_y.argmax(dim=1).int()

            # Run embedding over previous y state
            prev_y = self.embedding(prev_y)
            seq_dec_hiddens = prev_y.unsqueeze(0)
            for i in range(self.layers):
                positional_context = self.attention(context, dec_hiddens[j,i,], batch_size)
                hidden = self.decoders[i](torch.cat((seq_dec_hiddens[i,], positional_context), dim=1), dec_hiddens[j,i,],)
                # Add first sequence axis
                hidden = hidden.unsqueeze(0)
                seq_dec_hiddens = torch.cat((seq_dec_hiddens, hidden), dim=0)

            # Swap sequence and batch axes to apply linear transform, then swap back
            prev_output = self.linear(seq_dec_hiddens[-1]).unsqueeze(0)
            outputs = torch.cat((outputs, prev_output), dim=0)
            dec_hiddens = torch.cat((dec_hiddens, seq_dec_hiddens[1:].unsqueeze(0)), dim=0)

        # Move batch back to axis 0
        out_hiddens = dec_hiddens[1:,-1,:,:].swapaxes(0,1)
        out_output = outputs[1:,].swapaxes(0,1)
        return out_output, out_hiddens

def generate(sequence, prev_target, target, train_wrapper):
    pred, _ = model(sequence, prev_target[:,0].unsqueeze(1), target.shape[1])
    prompts = train_wrapper.decode_ids(sequence.cpu())
    texts = train_wrapper.decode_ids(torch.argmax(pred, dim=2).cpu())
    correct_texts = train_wrapper.decode_ids(target.cpu())

    displays = []
    for p, t, ct in zip(prompts, texts, correct_texts):
        displays.append(f"{p} | {ct} | {t}")
    return displays

In [6]:
from tqdm.auto import tqdm
import wandb

wandb.init(project="rnn-encoder", notes="Baseline performance one layer", name="baseline-one")

model = EncoderDecoder(SP_VOCAB_SIZE, hidden_units=512, layers=1).to(DEVICE)
loss_fn = nn.CrossEntropyLoss(ignore_index=train_wrapper.tokenizer.pad_token_id)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
wandb.watch(model, log_freq=100)

wandb: Currently logged in as: vikp. Use `wandb login --relogin` to force relogin


[]

In [ ]:
EPOCHS = 50
DISPLAY_BATCHES = 2

for epoch in range(EPOCHS):
    # Run over the training examples
    train_loss = 0
    for batch in tqdm(train):
        sequence = batch["en_ids"].to(torch.long)
        target = batch["es_ids"].to(torch.long)
        # Setup target properly
        prev_target = torch.roll(target, 1, -1)
        prev_target[:,0] = train_wrapper.tokenizer.bos_token_id

        optimizer.zero_grad()
        forced_target = prev_target
        # Alternate use of teacher forcing vs feeding back own inputs
        if np.random.randint(2) == 0:
            forced_target = prev_target[:,0].unsqueeze(1)
        pred, hidden = model(sequence.to(DEVICE), forced_target.to(DEVICE), max_sequence_len=target.shape[1])

        # Need to reshape pred to be batch * sequence, embedding_len to be compatible
        # Similar reshape with target to be batch * sequence vector of class indices
        loss = loss_fn(pred.reshape(-1, pred.shape[-1]), target.view(-1).to(DEVICE))
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    with torch.no_grad():
        mean_loss = train_loss / len(train)
        wandb.log({"train_loss": mean_loss})
        if epoch % 10 == 0:
            # Show text generated from training prompt as an example
            # Don't feed in all of the train y sequences, just the first token
            # The other y tokens will be predicted by the model and fed back in
            print(f"Epoch {epoch} train loss: {mean_loss}")
            sents = generate(sequence[:DISPLAY_BATCHES], prev_target[:DISPLAY_BATCHES], target[:DISPLAY_BATCHES], train_wrapper)
            for sent in sents:
                print(sent)

            # Compute validation loss.  Unless you have a lot of training data, the validation loss won't decrease.
            valid_loss = 0
            for batch in valid:
                # Only feed in the first token of the actual target
                sequence = batch["en_ids"].to(torch.long)
                target = batch["es_ids"].to(torch.long)
                # Setup target properly
                prev_target = torch.roll(target, 1, -1)
                prev_target[:,0] = train_wrapper.tokenizer.bos_token_id

                pred, hidden = model(sequence.to(DEVICE), prev_target[:,0].unsqueeze(1).to(DEVICE), max_sequence_len=target.shape[1])
                loss = loss_fn(pred.reshape(-1, pred.shape[-1]), target.view(-1).to(DEVICE))
                valid_loss += loss.item()

            mean_loss = valid_loss / len(valid)
            wandb.log({"valid_loss": mean_loss})
            print(f"Valid loss: {mean_loss}")
            sents = generate(sequence[:DISPLAY_BATCHES], prev_target[:DISPLAY_BATCHES], target[:DISPLAY_BATCHES], train_wrapper)
            for sent in sents:
                print(sent)

  0%|          | 0/33 [00:00<?, ?it/s]

Epoch 0 train loss: 6.7895668058684375
I am sorry I did not know it before; for I certainly would not have suffered you to give me particulars of a conversation which you ought not to have known yourself. | Lamento no haberlo sabido antes, pues de ninguna manera habría aceptado que me comunicara pormenores de una conversación que usted misma no debía conocer. | a que. que. que.. que. que.. que. que.. que. que.. que.. que.. que.. que.. que.. que.. que.. que.. que.
Edward was, of course, immediately convinced that nothing could have been more natural than Lucy's conduct, nor more self-evident than the motive of it. | Por supuesto, Edward se convenció de inmediato de que nada podía ser más natural que el comportamiento de Lucy, ni más palmario que sus motivos. | a. que.. que... que... que... que... que... que... que... que... que... que... que...
Valid loss: 6.344489574432373
Mrs. Dashwood was surprised only for a moment at seeing him; for his coming to Barton was, in her opinion, of all 

  0%|          | 0/33 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]